## Neo4j 연결

In [1]:
#Neo4j 서버 연결 확인 
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)

# Neo4j 서버에 연결
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")


# URI와 사용자 정보 설정
AUTH = (user, password)

# GraphDatabase에 드라이버 생성 및 연결 확인
with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    driver.verify_connectivity()
    print("Neo4j 서버와의 연결이 성공적으로 확인되었습니다.")


Neo4j 서버와의 연결이 성공적으로 확인되었습니다.


In [2]:
import json
from LegalGraphDB import LegalGraphDB

DATA_PATH = "/mnt/c/Users/Shic/legal_graph/results/1-2_subset/DCM_1-2_enforcement_main_clause.json"

#json 저장된 데이터 가져오기 
with open(DATA_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)
    print(f"Loaded {len(data)} data from {DATA_PATH}")


db = LegalGraphDB()

Loaded 1648 data from /mnt/c/Users/Shic/legal_graph/results/1-2_subset/DCM_1-2_enforcement_main_clause.json
## Neo4j (AuraDB) 서버와의 연결이 성공적으로 확인되었습니다.


## 노드 삽입

In [3]:
# #노드 생성 
# for (i, item) in enumerate(data):
#     for (j, node) in enumerate(item) : 
#         node = flatten_dict(item[j])
        
#         if "시행령" in node["metadata_document_title"]:
#             db.create_node("enforcement", node)
#         elif "법률" in node["metadata_document_title"]:
#             db.create_node("law", node)
#         elif "시행규칙" in node["metadata_document_title"]:
#             db.create_node("order", node)
        
# print(f"{data_path}의 데이터를 Neo4j에 nodes 생성 완료 ")


In [3]:
CONFIG_PATH = "/mnt/c/Users/Shic/legal_graph/codes/configs/config.json"

# config_file 가져오기
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)
    print(f"Loaded config data from {CONFIG_PATH}")

Loaded config data from /mnt/c/Users/Shic/legal_graph/codes/configs/config.json


In [4]:
from tqdm import tqdm
from utils import chunk_split
import copy

splitted_doc = 0

#노드 생성 
for raw in tqdm(data):
    chunks = chunk_split(raw['content'], config['chunk_size'], config['chunk_overlap'])
    if len(chunks) > 1:
        splitted_doc += 1

    for chunk_text in chunks:
        # metadata 따오고 내용 텍스트만 변환
        new_chunk = copy.deepcopy(raw)
        new_chunk['content'] = chunk_text

        if "시행령" in new_chunk["metadata"]["document_title"]:
            db.create_law_node("enforcement", new_chunk)
        elif "법률" in new_chunk["metadata"]["document_title"]:
            db.create_law_node("law", new_chunk)
        elif "시행규칙" in new_chunk["metadata"]["document_title"]:
            db.create_law_node("order", new_chunk)
        else:
            raise ValueError("## Check Data")
        
print(f"## {DATA_PATH}의 데이터를 Neo4j에 nodes 생성 완료 ")
print(f'## 청킹된 문서 크기 : {splitted_doc}')

100%|██████████| 1648/1648 [05:10<00:00,  5.30it/s]

## /mnt/c/Users/Shic/legal_graph/results/1-2_subset/DCM_1-2_enforcement_main_clause.json의 데이터를 Neo4j에 nodes 생성 완료 
## 청킹된 문서 크기 : 40


In [6]:
# Rule-based REFERS_TO 임시 구축 -> 해당하는 노드 찾아서 REFERS_TO로 구축하기

## UseCase1 : 제125조제3항 ~ 제125조제1항
usecase1 = [("law","제125조제3항", "law","제125조제2항"), ("law","제125조제2항", "law","제125조제1항")]

for f_type, f_index, t_type, t_index in usecase1:
    db.create_directed_relationship(f_type, f_index, t_type, t_index)

In [5]:
# Rule-based REFERS_TO 임시 구축 -> 해당하는 노드 찾아서 REFERS_TO로 구축하기

## UseCase2 (From, To) -> 2 hop까지 Subgraph 구축
usecase2 = [("law", "제71조", "law", "제7조제4항"), ("law","제249조의4제2항", "law","제7조제4항"),
            ("enforcement", "제68조제1항", "law", "제71조"), ("enforcement", "제68조제2항", "law", "제71조"), 
            ("enforcement", "제68조제3항", "law", "제71조"), ("enforcement", "제68조제4항", "law", "제71조"),
            ("enforcement", "제68조제5항", "law", "제71조"),
            ("enforcement", "제68조제2항", "law", "제249조의7제5항"),
            ("enforcement", "제68조제5항", "law", "제9조제5항"), ("enforcement", "제68조제5항", "enforcement", "제10조제1항"),
            ("enforcement", "제68조제5항", "law", "제124조제2항"), ("enforcement", "제68조제5항", "law", "제249조의4제2항"),
            ("enforcement", "제68조제5항", "law", "제119조제3항"), ("enforcement", "제68조제5항", "law", "제122조제1항"),
            ("enforcement", "제68조제5항", "law", "제165조의6제4항"), ("enforcement", "제68조제5항", "law", "제235조제6항"),
            ("enforcement", "제68조제5항", "law", "제55조"), ("enforcement", "제68조제5항", "enforcement", "제77조의6제2항"),
            ("enforcement", "제68조제5항", "enforcement", "제68조제1항"),("enforcement", "제68조제5항", "enforcement", "제77조의6제3항"),
            ("enforcement", "제68조제5항", "law", "제118조의9제1항"), ("enforcement", "제68조제5항", "law", "제130조제1항"),
            ("enforcement", "제68조제5항", "law", "제117조의10제6항"), ("enforcement", "제68조제5항", "law", "제118조의16제5항")
            ]


for f_type, f_index, t_type, t_index in usecase2:
    db.create_directed_relationship(f_type, f_index, t_type, t_index)


## Word Documentation 추가

- ex. 기업실사 -> 이걸로 연결될 수 있게끔
- 

## 노드 삭제

In [ ]:
db.delete_all_relationship()

In [ ]:
db.delete_all_nodes()